# Script Set-Up

In [ ]:
%matplotlib inline

In [ ]:
### Import Library
from scipy.io import loadmat
import os
import numpy as np
import mne
from matplotlib import pyplot as plt
import pandas as pd
import matplotlib.colors as mcolors
import math
import numpy.ma as ma

In [ ]:
os.chdir('../../') 

In [ ]:
baseline = 1

In [ ]:
if baseline == 0:
    baseline_name = "own_baseline"
elif baseline == 1:
    baseline_name = "pre_interval_baseline"
elif baseline == 2:
    baseline_name = "repo_baseline"

# Load data and convert into MNE Object
- Aids subsequent plotting and analyses

In [ ]:
ID_list = ['P1', 'P3', 'P4', 'P7', 'P10', 'P12', 'P15', 'P17', 'P19', 'P21', 'P22', 'P23', 'P24',
           'P26', 'P27', 'P28', 'P29', 'P30']

for ID in ID_list:
    
    ## Load data
    index = ID_list.index('P1')
    ID = ID_list[index]
    
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs_Arrays_CSD/" + "Response_Locked/" + ID + "_Response_Locked_All_CSD.mat"
    mat_dict = loadmat(fname)
    response_locked_epochs_MAT_CSD = mat_dict['response_locked_epochs_CSD']
    
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs_Arrays_CSD/" + "Encode/" + ID + "_Encode_All_CSD"
    encoding_mat_dict = loadmat(fname)
    encoding_epochs_MAT_CSD = encoding_mat_dict['encode_epochs_CSD']
    
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs_Arrays_CSD/" + "Reproduction/" + ID + "_Reproduction_All_CSD"
    reproduction_mat_dict = loadmat(fname)
    repo_epochs_MAT_CSD = reproduction_mat_dict['repo_epochs_CSD']
    
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs_Arrays_CSD/" + "Offset/" + ID + "_Offset_All_CSD"
    offset_mat_dict = loadmat(fname)
    offset_epochs_MAT_CSD = offset_mat_dict['offset_epochs_CSD']
    
    ## Setting Up Epochs Array for MNE
    # Need to change dimensions to form of n_epochs, n_channels, n_samples
    # Need to convert from uV/m^2 to V/m^2
    
    # Response-Locked Epochs
    response_locked_epochs_MAT_CSD = np.moveaxis(response_locked_epochs_MAT_CSD, -1, 0) # change shape of epochs
    response_locked_epochs_MAT_CSD = response_locked_epochs_MAT_CSD / 1e6 # convert from uV/m^2 to V/m^2
    
    # Encoding Epochs
    encoding_epochs_MAT_CSD = np.moveaxis(encoding_epochs_MAT_CSD, -1, 0) # change shape of epochs
    encoding_epochs_MAT_CSD = encoding_epochs_MAT_CSD / 1e6 # convert from uV/m^2 to V/m^2
    
    # Reproduction Epochs 
    repo_epochs_MAT_CSD = np.moveaxis(repo_epochs_MAT_CSD, -1, 0) # change shape of epochs
    repo_epochs_MAT_CSD = repo_epochs_MAT_CSD / 1e6 # convert from uV/m^2 to V/m^2

    # Offset Epochs 
    offset_epochs_MAT_CSD = np.moveaxis(offset_epochs_MAT_CSD, -1, 0) # change shape of epochs
    offset_epochs_MAT_CSD = offset_epochs_MAT_CSD / 1e6 # convert from uV/m^2 to V/m^2
    
    ## Loading Original Epochs Object to get metadata and info
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs/" + "Response_Locked/" + ID + "_RL_All_epo.fif"
    RL_epochs = mne.read_epochs(fname = fname, preload = True)

    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs/" + "Encode/" + ID + "_Encode_All_epo.fif"
    encoding_epochs = mne.read_epochs(fname = fname, preload = True)
    
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs/" + "Reproduction/" + ID + "_Reproduction_All_epo.fif"
    repo_epochs = mne.read_epochs(fname = fname, preload = True)
    
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + "pre_interval_baseline/" + "Epochs/" + "Offset/" + ID + "_Offset_All_epo.fif"
    offset_epochs = mne.read_epochs(fname = fname, preload = True)
    
    info = offset_epochs.average().info # get info from evoked rather than epochs because 128 channels only now that CSD has been applied
    print(info)
    
    ## Create Response-Locked Epochs
    metadata = RL_epochs.metadata
    events = RL_epochs.events
     
    event_dict = {'Response/600':201,'Response/650':202,'Response/700':203,'Response/750':204,'Response/800':205,
                   'Response/850':206,'Response/900':207,'Response/950':208,'Response/1000':209}
    
    RL_epochs_MAT_CSD = mne.EpochsArray(data = response_locked_epochs_MAT_CSD, info = info, tmin = -1.1, events = events,
                                        event_id = event_dict, metadata = metadata, baseline = None, reject = None)
    
    ## Create Encoding Epochs
    metadata = encoding_epochs.metadata
    events = encoding_epochs.events
    
    event_dict = {'Encode/600':51,'Encode/650':52,'Encode/700':53,'Encode/750':54,'Encode/800':55,
                          'Encode/850':56,'Encode/900':57,'Encode/950':58,'Encode/1000':59}
    
    encoding_epochs_MAT_CSD = mne.EpochsArray(data = encoding_epochs_MAT_CSD, info = info, tmin = -0.1, events = events,
                                              event_id = event_dict, metadata = metadata, baseline = None, reject = None)
    
    ## Create Reproduction Epochs
    metadata = repo_epochs.metadata
    events = repo_epochs.events
    
    event_dict = {'Reproduction/600':151,'Reproduction/650':152,'Reproduction/700':153,'Reproduction/750':154,
                  'Reproduction/800':155, 'Reproduction/850':156,'Reproduction/900':157,'Reproduction/950':158,
                   'Reproduction/1000':159}
     
    repo_epochs_MAT_CSD = mne.EpochsArray(data = repo_epochs_MAT_CSD, info = info, tmin = -0.1, events = events,
                                           event_id = event_dict, metadata = metadata, baseline = None, reject = None)
    
    ## Create Offset Epochs
    metadata = offset_epochs.metadata
    events = offset_epochs.events
    
    event_dict = {'End_Encode/600':101,'End_Encode/650':102,'End_Encode/700':103,
                       'End_Encode/750':104,'End_Encode/800':105,'End_Encode/850':106,
                       'End_Encode/900':107,'End_Encode/950':108,'End_Encode/1000':109}
    
    offset_epochs_MAT_CSD = mne.EpochsArray(data = offset_epochs_MAT_CSD, info = info, tmin = offset_epochs.tmin, events = events,
                                          event_id = event_dict, metadata = metadata, baseline = None, reject = None)
    
    ## Save Files
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs/" + "Response_Locked/" + ID + "_RL_All_MAT_CSD_epo.fif"
    RL_epochs_MAT_CSD.save(fname = fname, overwrite = True)
    
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs/" + "Encode/" + ID + "_Encode_All_MAT_CSD_epo.fif"
    encoding_epochs_MAT_CSD.save(fname = fname, overwrite = True)
    
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + baseline_name + "/Epochs/" + "Reproduction/" + ID + "_Reproduction_All_MAT_CSD_epo.fif"
    repo_epochs_MAT_CSD.save(fname = fname, overwrite = True)
    
    fname = "Time_Reproduction_Data/" + "Alternate_Artifact_Rejection/" + "pre_interval_baseline/" + "Epochs/" + "Offset/" + ID + "_Offset_All_MAT_CSD_epo.fif"
    offset_epochs_MAT_CSD.save(fname = fname, overwrite = True)